In [52]:
from random import randint
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

In [53]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 14 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
ad_campaign_hit                38829 non-null bool
cant_lead                      38829 non-null float64
lead                           38829 non-null bool
cant_brand_listing             38829 non-null float64
cant_visited_site              38829 non-null float64
cant_generic_listing           38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
dtypes: bool(2), float64(11), int64(1)
memory usage: 3.9+ MB


In [54]:
df2 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')


######################################################################################


df5 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df8 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/channel_device.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('data/check.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('data/view_sku.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('data/event_por_hora.csv', low_memory=False, index_col='person')


df_unidos = df1.join(df2).fillna(0)
"""df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
#df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
#df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)
df_unidos = df_unidos.join(df13).fillna(0)
df_unidos = df_unidos.join(df14).fillna(0)
df_unidos = df_unidos.join(df15).fillna(0)
df_unidos = df_unidos.join(df16).fillna(0)"""

df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 19 columns):
cant_conversions               38829 non-null float64
cant_checkouts                 38829 non-null float64
cant_viewed_product            38829 non-null float64
cant_searched_product          38829 non-null float64
cant_ad_campaign_hit           38829 non-null float64
ad_campaign_hit                38829 non-null bool
cant_lead                      38829 non-null float64
lead                           38829 non-null bool
cant_brand_listing             38829 non-null float64
cant_visited_site              38829 non-null float64
cant_generic_listing           38829 non-null float64
total_sesiones                 38829 non-null int64
promedio_eventos_por_sesion    38829 non-null float64
proprosion                     38829 non-null float64
timestamp                      38829 non-null int64
min_timestamp                  38829 non-null int64
sum_timestamp                  388

In [55]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_test = df_unidos.join(labels, how= "inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, ad93850f to d8d6b0da
Data columns (total 20 columns):
cant_conversions               19414 non-null float64
cant_checkouts                 19414 non-null float64
cant_viewed_product            19414 non-null float64
cant_searched_product          19414 non-null float64
cant_ad_campaign_hit           19414 non-null float64
ad_campaign_hit                19414 non-null bool
cant_lead                      19414 non-null float64
lead                           19414 non-null bool
cant_brand_listing             19414 non-null float64
cant_visited_site              19414 non-null float64
cant_generic_listing           19414 non-null float64
total_sesiones                 19414 non-null int64
promedio_eventos_por_sesion    19414 non-null float64
proprosion                     19414 non-null float64
timestamp                      19414 non-null int64
min_timestamp                  19414 non-null int64
sum_timestamp                  194

In [56]:
features = df_test.columns.tolist()
features.remove('label')
print(features)

['cant_conversions', 'cant_checkouts', 'cant_viewed_product', 'cant_searched_product', 'cant_ad_campaign_hit', 'ad_campaign_hit', 'cant_lead', 'lead', 'cant_brand_listing', 'cant_visited_site', 'cant_generic_listing', 'total_sesiones', 'promedio_eventos_por_sesion', 'proprosion', 'timestamp', 'min_timestamp', 'sum_timestamp', 'prom_timestamp', 'ult_view']


In [57]:
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [58]:
param_grid = { 
    'max_depth' : [2,3,4,5,6,7,8,9,10],
    
} 

In [59]:
from sklearn.model_selection import GridSearchCV 
clf = tree.DecisionTreeClassifier()
CV_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv= 5, scoring='roc_auc', n_jobs=-1, verbose=10) 
CV_clf.fit(X_train, Y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=2 .....................................................
[CV] max_depth=2 .....................................................
[CV] ................. max_depth=2, score=0.72824246529, total=   0.1s
[CV] ................ max_depth=2, score=0.757828270612, total=   0.1s
[CV] max_depth=2 .....................................................
[CV] ................ max_depth=2, score=0.745371205158, total=   0.1s
[CV] max_depth=2 .....................................................
[CV] ................ max_depth=2, score=0.764017305206, total=   0.1s
[CV] max_depth=2 .....................................................
[CV] ................ max_depth=2, score=0.744999512904, total=   0.1s
[CV] max_depth=3 .....................................................
[CV] ................ max_depth=3, score=0.801363130605, total=   0.1s


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.0s


[CV] max_depth=3 .....................................................
[CV] ................ max_depth=3, score=0.814154436826, total=   0.1s
[CV] max_depth=3 .....................................................
[CV] ................ max_depth=3, score=0.814748153979, total=   0.1s
[CV] max_depth=3 .....................................................
[CV] ................ max_depth=3, score=0.808084028553, total=   0.1s
[CV] max_depth=3 .....................................................
[CV] ................ max_depth=3, score=0.802747002143, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................ max_depth=4, score=0.805457580009, total=   0.1s


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.8s


[CV] max_depth=4 .....................................................
[CV] ................ max_depth=4, score=0.822254278062, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................ max_depth=4, score=0.818060875799, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................. max_depth=4, score=0.80045543511, total=   0.1s
[CV] max_depth=4 .....................................................
[CV] ................ max_depth=4, score=0.815990488336, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=5, score=0.799188366662, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=5, score=0.826596109613, total=   0.1s
[CV] max_depth=5 .....................................................
[CV] ................ max_depth=5, score=0.825189879541, total=   0.1s


...

[CV] max_depth=9 .....................................................
[CV] ................ max_depth=8, score=0.773100545548, total=   0.2s
[CV] max_depth=9 .....................................................
[CV] ................ max_depth=9, score=0.730210527705, total=   0.2s
[CV] max_depth=9 .....................................................
[CV] ................ max_depth=9, score=0.738304871559, total=   0.3s
[CV] max_depth=9 .....................................................
[CV] ................ max_depth=9, score=0.741503247853, total=   0.4s
[CV] max_depth=9 .....................................................
[CV] ................ max_depth=9, score=0.663533751351, total=   0.3s
[CV] max_depth=10 ....................................................
[CV] ................ max_depth=9, score=0.707583869139, total=   0.4s
[CV] max_depth=10 ....................................................
[CV] ............... max_depth=10, score=0.673033366904, total=   0.3s
[CV] m

[Parallel(n_jobs=-1)]: Done  43 out of  45 | elapsed:    8.5s remaining:    0.4s


[CV] ............... max_depth=10, score=0.670723958057, total=   0.2s


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    8.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=10)

In [60]:
CV_clf.best_params_ #aca te dice el mejor hiperparametro

{'max_depth': 5}

In [61]:
clf = tree.DecisionTreeClassifier(max_depth= 5)
clf.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [62]:
clf.score(X_test,Y_test)

0.94437290754571213

In [63]:
Y_pred = clf.predict(X_test)
print(roc_auc_score(Y_test, Y_pred))

0.510566628804


In [64]:
Y_truths = Y_test
print(classification_report(Y_truths, Y_pred))

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      3671
          1       0.36      0.02      0.04       212

avg / total       0.91      0.94      0.92      3883



In [65]:
print(confusion_matrix(Y_truths, Y_pred))

[[3662    9]
 [ 207    5]]


In [66]:
feature_importances = pd.DataFrame(clf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

In [67]:
feature_importances.head(60)

,importance
cant_checkouts,0.395484
sum_timestamp,0.156838
timestamp,0.143643
cant_conversions,0.097797
cant_viewed_product,0.060144
promedio_eventos_por_sesion,0.043787
ult_view,0.042230
prom_timestamp,0.022755
cant_brand_listing,0.016030
cant_generic_listing,0.010123


In [68]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [69]:
kaggle_pred = clf.predict_proba(df_events)
kaggle_pred
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [70]:
df_submit.label.mean()

0.050185860951595176

In [71]:
df_submit.to_csv('submit/submit_clf.csv')

In [72]:
df_submit['label']


person
4886f805    0.005950
0297fc1e    0.057402
2d681dd8    0.005950
cccea85e    0.057402
4c8a8b93    0.057402
29ebb414    0.005950
3dc1950f    0.057402
8ea4c165    0.057402
d8cfe234    0.034921
d6bc64df    0.057402
ec9c4059    0.034921
d21b8e6b    0.005950
2a724d87    0.005950
686c49c9    0.195248
a08d42ea    0.034921
c98f5cf1    0.057402
d614c608    0.005950
e45acd53    0.032258
7d876393    0.057402
f5af843f    0.057402
5a724794    0.057402
00091926    0.057402
55d1e0ee    0.034921
f87be219    0.005950
49c19e32    0.034921
bb78c182    0.005950
e2bfe05f    0.005950
0e9d0ae2    0.005950
cb68850c    0.005950
f30ef764    0.005950
              ...   
523c7e69    0.096561
d2e564cb    0.195248
eb27e544    0.195248
b32e7113    0.096561
a65f2cf0    0.096561
a70b9f00    0.295385
a161fd76    0.096561
e9a4d3a8    0.096561
24f53ba2    0.096561
39cf8fa0    0.096561
9ceab28a    0.096561
f85da107    0.096561
154d2935    0.096561
3adf7ca4    0.195248
2e89874a    0.096561
ef4e52ab    0.096561
85e0f6